### DataQuality

In [0]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline
import xgboost as xgb
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


In [0]:
spark.conf.set("###",dbutils.secrets.get(scope = "###", key = "###"))

In [0]:
filenames=dbutils.fs.ls("Retraining/FeatureEngineeringData/")
filelist=[file.name for file in filenames if 'FeatureEngineeringData_' in file.name]
filedates=[name.split('_')[1] for name in filelist]
filedates=[name.split('.')[0] for name in filedates]

In [0]:
import pandas as pd
pdfiledates = pd.Series(filedates)
pdfiledates = pd.to_datetime(pdfiledates, infer_datetime_format=True)

In [0]:
maxdate=str(pdfiledates.max()).replace(' ','T').replace(':','H',1).replace(':','m',1)+'s'

In [0]:
dffeatureEngTrain=spark.read.parquet("FeatureEngineeringData/FeatureEngineeringData_"+maxdate+".parquet")

In [0]:
dfk = dffeatureEngTrain.toPandas()

In [0]:
dfk = dfk[[
'0_month_ago_CountOfPaymentStatusPaidOff',
'1_to_4_avg_CountOfPaymentStatusPaidOff',
'0_month_ago_CountOfInstrumentStatusClosed',
'1_to_4_avg_CountOfInstrumentStatusClosed',
'0_month_ago_CountOfInstrumentStatusOpen',
'1_to_4_avg_CountOfInstrumentStatusOpen',
'0_month_ago_CountOfPaymentStatusChargedOff',
'0_month_ago_CountOfPaymentStatusChargedOffPaidinFull',
'1_to_4_avg_CountOfPaymentStatusChargedOffPaidinFull',
'0_month_ago_CountOfInstrumentConditionInactive',
'0_month_ago_CountOfInstrumentConditionBankruptcy',
'0_month_ago_CountOfInstrumentConditionClosedByInstitution',
'0_month_ago_CountOfPaymentStatusNegativeShareBalance',
'1_to_4_avg_CountOfInstrumentConditionInactive',
'1_to_4_avg_CountOfInstrumentConditionBankruptcy',
'1_to_4_avg_CountOfPaymentStatusNegativeShareBalance',
'1_to_4_avg_CountOfInstrumentConditionClosedByInstitution',
'0_to_1_pct_change_Net_AmountShare',
'1_to_2_pct_change_Net_AmountShare',
'2_to_3_pct_change_Net_AmountShare',
'3_to_4_pct_change_Net_AmountShare',
'0_to_1_pct_change_Net_AmountLoan',
'1_to_2_pct_change_Net_AmountLoan',
'2_to_3_pct_change_Net_AmountLoan',
'3_to_4_pct_change_Net_AmountLoan',
'0_to_1_pct_change_ProdAmountPOETransactional',
'1_to_2_pct_change_ProdAmountPOETransactional',
'2_to_3_pct_change_ProdAmountPOETransactional',
'3_to_4_pct_change_ProdAmountPOETransactional',
'0_to_1_pct_change_ProdAmountPOESavings',
'1_to_2_pct_change_ProdAmountPOESavings',
'2_to_3_pct_change_ProdAmountPOESavings',
'3_to_4_pct_change_ProdAmountPOESavings',
'0_month_ago_ProdCountPOETransactional',
'0_month_ago_ProdCountPOEWealth',
'0_month_ago_ProdCountPOERealEstate',
'0_month_ago_ProdCountPOESavings',
'0_month_ago_ProdCountPOELoans',
'0_month_ago_POELoans',
'0_month_ago_POERealEstate',
'0_month_ago_POESavings',
'0_month_ago_POETransactional',
'0_month_ago_POEWealth',
'0_month_ago_ProdAmountPOETransactional',
'0_month_ago_ProdAmountPOEWealth',
'0_month_ago_ProdAmountPOERealEstate',
'0_month_ago_ProdAmountPOESavings',
'0_month_ago_ProdAmountPOELoans',
'1_to_2_avg_ProdAmountPOEWealth',
'3_to_4_avg_ProdAmountPOEWealth',
'1_to_2_avg_ProdAmountPOETransactional',
'3_to_4_avg_ProdAmountPOETransactional',
'1_to_2_avg_ProdAmountPOERealEstate',
'3_to_4_avg_ProdAmountPOERealEstate',
'1_to_2_avg_ProdAmountPOESavings',
'3_to_4_avg_ProdAmountPOESavings',
'1_to_2_avg_ProdAmountPOELoans',
'3_to_4_avg_ProdAmountPOELoans',
'1_to_2_avg_Net_AmountLoan',
'3_to_4_avg_Net_AmountLoan',
'0_month_ago_Net_AmountLoan',
'1_to_2_avg_Net_AmountShare',
'3_to_4_avg_Net_AmountShare',
'0_month_ago_Net_AmountShare',
'1_to_4_avg_Net_CountLoan',
'0_month_ago_Net_CountLoan',
'1_to_4_avg_Net_CountShare',
'0_month_ago_Net_CountShare',
'1_to_4_avg_ProdCountPOETransactional',
'1_to_4_avg_ProdCountPOEWealth',
'1_to_4_avg_ProdCountPOERealEstate',
'1_to_4_avg_ProdCountPOESavings',
'1_to_4_avg_ProdCountPOELoans',
'voting_1_4POEWealth',
'voting_1_4POELoans',
'voting_1_4POERealEstate',
'voting_1_4POESavings',
'voting_1_4POETransactional',
'DaysSinceEngaged',
'1_to_4_avg_NoOfTransactionsBranch',
'1_to_4_avg_NoOfTransactionsOperations',
'1_to_4_avg_NoOfTransactionsContactCenter',
'1_to_4_avg_NoOfTransactionsIndirectLending',
'1_to_4_avg_NoOfTransactionsSharedBranch',
'1_to_4_avg_NoOfTransactionsMobileOnlineBanking',
'1_to_4_avg_NoOfTransactionsSystemGenerated',
'1_to_4_avg_NoOfTransactionsElectronicClearing',
'1_to_4_avg_NoOfTransactionsMerchantSign',
'1_to_4_avg_NoOfTransactionsLockbox',
'0_month_ago_NoOfTransactionsBranch',
'0_month_ago_NoOfTransactionsOperations',
'0_month_ago_NoOfTransactionsContactCenter',
'0_month_ago_NoOfTransactionsIndirectLending',
'0_month_ago_NoOfTransactionsSharedBranch',
'0_month_ago_NoOfTransactionsMobileOnlineBanking',
'0_month_ago_NoOfTransactionsSystemGenerated',
'0_month_ago_NoOfTransactionsElectronicClearing',
'0_month_ago_NoOfTransactionsMerchantSign',
'0_month_ago_NoOfTransactionsLockbox',
'1_to_2_avg_HouseholdIdentifier_count',
'0_month_ago_HouseholdIdentifier_count',
'voting_1_4HasFPSInvestment',
'voting_1_4HasDirectDeposit',
'voting_1_4HasOnlineBanking',
'voting_1_4HasNoMarketing',
'voting_1_4HasCheckingRewards',
'0_month_ago_HasCheckingRewards',
'0_month_ago_HasDirectDeposit',
'0_month_ago_HasOnlineBanking',
'0_month_ago_HasNoMarketing',
'0_month_ago_HasFPSInvestment',
'0_month_ago_HasNoMarketingMail',
'TARGET']]

In [0]:
duplicate_columns = dfk.columns[dfk.columns.duplicated()]
duplicate_columns

In [0]:
if len(duplicate_columns) > 0: 
  raise Exception("Duplicate columns exist!")

In [0]:
percent_missing = dfk.isnull().sum() * 100 / len(dfk)
missing_value_df = pd.DataFrame({'column_name': dfk.columns,
                                 'percent_missing': percent_missing})

In [0]:
missing_value_df['greaterthan7'] = missing_value_df['percent_missing'] > 7.0

In [0]:
display(missing_value_df)

In [0]:
if missing_value_df['greaterthan7'].any() == True:
   raise Exception("Missing values greater than 7 percent!")

In [0]:
dfk = dfk.apply(pd.to_numeric)

In [0]:
df4 = dfk[dfk.isin([np.inf, -np.inf]).any(1)]
df4

In [0]:
if len(df4) > 0: 
  raise Exception("Infinitive values exist!")
 